##*Extracción, Transformación y Carga (ETL) del archivo "australian_users_items.json"*

### Importamos librerías

In [8]:
import json
import ast
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

### Extraemos y exploramos la data

In [9]:
row = []  # creamos una lista vacia para ir agregando las filas del archivo json

with open("users_items.json\\australian_users_items.json", encoding="utf-8") as file:
    for line in file:
        # Utilizamos ast.literal_eval para interpretar la línea como expresión literal de Python
        # Luego, convertimos la expresión evaluada a formato JSON utilizando json.dumps
        row.append(json.loads(json.dumps(ast.literal_eval(line))))

# Creamos un dataframe con la lista de diccionarios
items = pd.DataFrame(row)
items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [10]:
items.shape # vemos la cantidad de filas y columnas

(88310, 5)

### Transformamos el dataset

La columna "items" contiene datos anidados en forma de una lista de diccionarios. En este conjunto de datos, se aplica el método de normalización, utilizando la columna "items" como columna de referencia. De esta manera, se extraen las claves de los diccionarios para crear columnas individuales.

In [11]:
# normalizamos la columna 'items'
normalizado = pd.json_normalize(row, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )
normalizado

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [12]:
tipo_data = {"columna":[],"tipos_de_datos":[]} # generamos un diccionario vacio para ir almacenando lo que genere el bucle

for columna in normalizado.columns: # recorremos las columnas del dataframe 
    tipo_data["columna"].append(columna) # agregamos el nombre de la columna
    tipo_data["tipos_de_datos"].append(normalizado[columna].apply(type).unique()) # agregamos los tipos de datos que hay en la columna

analisis= pd.DataFrame(tipo_data) # creamos un dataframe con el diccionario
analisis

,columna,tipos_de_datos
0,item_id,[<class 'str'>]
1,item_name,[<class 'str'>]
2,playtime_forever,[<class 'int'>]
3,playtime_2weeks,[<class 'int'>]
4,steam_id,[<class 'str'>]
5,items_count,[<class 'int'>]
6,user_id,[<class 'str'>]
7,user_url,[<class 'str'>]


In [13]:
normalizado.info() # vemos los tipos de datos de cada columna

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          object
 5   items_count       object
 6   user_id           object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


 En este dataframe, optamos por realizar la búsqueda y eliminación de valores nulos después de la normalización. Esta elección se basa en la amplia información contenida en la columna anidada "items". El objetivo es evitar la pérdida de datos sin realizar un análisis exhaustivo previo.

In [14]:
duplicados = normalizado.loc[normalizado.duplicated()] # buscamos los duplicados
duplicados

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
164294,20,Team Fortress Classic,5,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164295,50,Half-Life: Opposing Force,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164296,70,Half-Life,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164297,130,Half-Life: Blue Shift,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164298,220,Half-Life 2,198,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
...,...,...,...,...,...,...,...,...
4898223,213670,South Park™: The Stick of Truth™,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


### Analizamos las filas con datos duplicados y se eliminan

In [15]:
normalizado = normalizado.drop_duplicates(keep='first')
normalizado

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [16]:
normalizado['playtime_forever'].value_counts()

playtime_forever
0        1847730
1         101586
2          34391
3          31530
4          29127
          ...   
76541          1
64676          1
53328          1
44029          1
34753          1
Name: count, Length: 48861, dtype: int64

In [17]:
# Filtramos y mantenemos solo las filas donde playtime_forever no es igual a 0
normalizado = normalizado[normalizado['playtime_forever'] != 0]

# Restablecemos los índices después de filtrar
normalizado = normalizado.reset_index(drop=True)

In [18]:
normalizado['playtime_forever'].value_counts()

playtime_forever
1        101586
2         34391
3         31530
4         29127
5         27612
          ...  
76541         1
64676         1
53328         1
44029         1
34753         1
Name: count, Length: 48860, dtype: int64

Dado que los datos en la columna "playtime_forever" están en minutos, se procede a convertirlos a horas, generando una nueva columna para esta transformación.

In [19]:
# Creamos la columna 'playtime_forever_hours' que es el resultado de dividir 'playtime_forever' por 60 y redondeando a 2 decimales
normalizado['playtime_hours'] = round(normalizado['playtime_forever']/60,2)

In [20]:
normalizado['playtime_hours'].value_counts() # vemos la cantidad de horas jugadas

playtime_hours
0.02       101586
0.03        34391
0.05        31530
0.07        29127
0.08        27612
            ...  
1275.68         1
1077.93         1
888.80          1
733.82          1
579.22          1
Name: count, Length: 48860, dtype: int64

 Continuando en la misma línea, y considerando que las columnas 'playtime_forever' y 'playtime_2weeks' se vuelven innecesarias, procederemos a eliminarlas para reducir aún más el tamaño del dataset.

In [21]:
# Eliminamos las columnas 'playtime_2weeks' y 'playtime_2weeks' por no ser de utilidad para el análisis
normalizado = normalizado.drop(['playtime_2weeks','playtime_forever'], axis=1)

In [22]:
normalizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3246375 entries, 0 to 3246374
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   item_id         object 
 1   item_name       object 
 2   steam_id        object 
 3   items_count     object 
 4   user_id         object 
 5   user_url        object 
 6   playtime_hours  float64
dtypes: float64(1), object(6)
memory usage: 173.4+ MB


### Columna 'user_url'

 La columna 'user_url' contiene la url del usuario en la plataforma Steam. Esta columna no es necesaria para el análisis y se eliminará.

In [23]:
# Eliminamos la columna 'usr_url'
normalizado = normalizado.drop(['user_url'], axis=1)

 Al igual que en la etapa anterior del proceso ETL, procedemos a guardar el dataframe en formato Parquet. Esta elección busca maximizar la eficiencia en las operaciones de lectura y escritura, además de optimizar el almacenamiento de datos.SS

In [24]:
normalizado = normalizado.copy() # copiamos el dataframe para no modificar el original

In [25]:
normalizado.to_csv("australian_users_items.csv", index=False, encoding="utf-8") # guardamos el dataframe en un archivo csv

In [26]:
normalizado = pd.read_csv("australian_users_items.csv") # leemos el archivo csv

tabla = pa.Table.from_pandas(normalizado) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"australian_users_items.parquet") # guardamos la tabla en un archivo parquet